In [19]:
import time
from datetime import datetime, timedelta
from binance.client import Client
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np


class CryptoFlowAnalyzer:
    def __init__(self,
                 api_key,
                 api_secret,
                 quote_asset='USDC',
                 timeframe='1h',
                 max_symbols=50,
                 threads=10,
                 small_thresh=10,
                 mid_thresh=100):
        
        self.client = Client(api_key, api_secret)
        self.quote_asset = quote_asset
        self.timeframe = timeframe
        self.max_symbols = max_symbols
        self.threads = threads
        self.small_thresh = small_thresh
        self.mid_thresh = mid_thresh

        self.timeframe_map = {
            '30m': timedelta(minutes=30),
            '1h': timedelta(hours=1),
            '1d': timedelta(days=1),
        }

        assert self.timeframe in self.timeframe_map, "Timeframe must be one of '30m', '1h', '1d'"
        assert small_thresh < mid_thresh, "small_thresh must be less than mid_thresh"

    def get_start_time_ms(self):
        delta = self.timeframe_map[self.timeframe]
        return int((datetime.utcnow() - delta).timestamp() * 1000)

    def fetch_all_symbols(self):
        exchange_info = self.client.get_exchange_info()
        return [
            s['symbol'] for s in exchange_info['symbols']
            if s['quoteAsset'] == self.quote_asset and s['status'] == 'TRADING'
        ][:self.max_symbols]

    def fetch_price_data(self, symbol):
        try:
            ticker = self.client.get_ticker(symbol=symbol)
            price_change_pct = float(ticker.get('priceChangePercent', 0))
            last_price = float(ticker.get('lastPrice', 0))
            return price_change_pct, last_price
        except Exception as e:
            print(f"[!] Error fetching price data for {symbol}: {e}")
            return 0.0, 0.0

    def fetch_agg_trades(self, symbol, start_time_ms, end_time_ms):
        trades = []
        while True:
            try:
                batch = self.client.get_aggregate_trades(
                    symbol=symbol,
                    startTime=start_time_ms,
                    endTime=end_time_ms,
                    limit=1000
                )
                if not batch:
                    break
                trades.extend(batch)

                last_trade_time = batch[-1]['T']
                if last_trade_time >= end_time_ms:
                    break
                start_time_ms = last_trade_time + 1
                time.sleep(0.05)  # avoid rate limits
            except Exception as e:
                print(f"[!] Error fetching trades for {symbol}: {e}")
                break
        return trades

    def fetch_volatility(self, symbol):
        # Fetch last n candles according to timeframe to calculate volatility
        interval = self.timeframe
        limit = 20  # number of candles to analyze volatility, ~20 periods
        
        try:
            klines = self.client.get_klines(symbol=symbol, interval=interval, limit=limit)
            closes = [float(k[4]) for k in klines]
            if len(closes) < 2:
                return 0.0
            volatility = np.std(closes) / np.mean(closes)  # coefficient of variation
            return volatility
        except Exception as e:
            print(f"[!] Error fetching volatility for {symbol}: {e}")
            return 0.0

    def categorize_order_size(self, trade_value):
        if trade_value < self.small_thresh:
            return 'small'
        elif trade_value < self.mid_thresh:
            return 'mid'
        else:
            return 'large'

    def analyze_agg_trades(self, trades):
        counts = {'buy': {'small': 0, 'mid': 0, 'large': 0}, 'sell': {'small': 0, 'mid': 0, 'large': 0}}
        values = {'buy': {'small': 0, 'mid': 0, 'large': 0}, 'sell': {'small': 0, 'mid': 0, 'large': 0}}
        total_buy = 0
        total_sell = 0

        for trade in trades:
            price = float(trade['p'])
            qty = float(trade['q'])
            value = price * qty
            side = 'sell' if trade['m'] else 'buy'
            size = self.categorize_order_size(value)

            counts[side][size] += 1
            values[side][size] += value

            if side == 'buy':
                total_buy += value
            else:
                total_sell += value

        return counts, values, total_buy, total_sell

    def process_symbol(self, symbol, start_time_ms, end_time_ms):
        trades = self.fetch_agg_trades(symbol, start_time_ms, end_time_ms)
        if not trades:
            return None

        counts, values, total_buy, total_sell = self.analyze_agg_trades(trades)

        total_volume = total_buy + total_sell
        net_flow_pct = (total_buy - total_sell) / total_volume if total_volume > 0 else 0
        buy_sell_ratio = total_buy / total_sell if total_sell > 0 else float('inf')
        price_change_pct, current_price = self.fetch_price_data(symbol)
        volatility = self.fetch_volatility(symbol)

        return {
            'ticker': symbol,
            'current_price': round(current_price, 6),
            'small_buy_orders': counts['buy']['small'],
            'mid_buy_orders': counts['buy']['mid'],
            'large_buy_orders': counts['buy']['large'],
            'small_sell_orders': counts['sell']['small'],
            'mid_sell_orders': counts['sell']['mid'],
            'large_sell_orders': counts['sell']['large'],
            'small_buy_value': round(values['buy']['small'], 2),
            'mid_buy_value': round(values['buy']['mid'], 2),
            'large_buy_value': round(values['buy']['large'], 2),
            'small_sell_value': round(values['sell']['small'], 2),
            'mid_sell_value': round(values['sell']['mid'], 2),
            'large_sell_value': round(values['sell']['large'], 2),
            'total_buy_volume': round(total_buy, 2),
            'total_sell_volume': round(total_sell, 2),
            'buy_sell_diff': round(total_buy - total_sell, 2),
            'total_volume': round(total_volume, 2),
            'net_flow_pct': round(net_flow_pct, 4),
            'buy_sell_ratio': round(buy_sell_ratio, 4),
            'price_change_pct': round(price_change_pct, 2),
            'volatility': round(volatility, 6)
        }

    def create_summary(self):
        symbols = self.fetch_all_symbols()
        print(f"🔍 Analyzing {len(symbols)} symbols on {self.timeframe} timeframe...")

        start_time_ms = self.get_start_time_ms()
        end_time_ms = int(datetime.utcnow().timestamp() * 1000)

        results = []
        with ThreadPoolExecutor(max_workers=self.threads) as executor:
            futures = {
                executor.submit(self.process_symbol, symbol, start_time_ms, end_time_ms): symbol
                for symbol in symbols
            }
            for i, future in enumerate(as_completed(futures), 1):
                symbol = futures[future]
                try:
                    result = future.result()
                    if result:
                        results.append(result)
                except Exception as e:
                    print(f"[!] Error processing {symbol}: {e}")
                print(f"[{i}/{len(symbols)}] Processed {symbol}")

        df = pd.DataFrame(results)
        if df.empty:
            print("No data processed.")
            return df

        # Normalize scores (rank percentile)
        df['net_flow_score'] = df['net_flow_pct'].rank(pct=True)
        df['price_change_score'] = df['price_change_pct'].rank(pct=True)
        df['volume_score'] = df['total_volume'].rank(pct=True)
        df['volatility_score'] = 1 - df['volatility'].rank(pct=True)  # less volatility = higher score
        # Balanced buy/sell ratio score, best near 1
        df['balanced_ratio_score'] = 1 - abs(df['buy_sell_ratio'] - 1)
        df['balanced_ratio_score'] = df['balanced_ratio_score'].clip(lower=0)  # no negative scores

        # Combine scores with weights (adjust weights as you want)
        df['sirifi_ranking_score'] = (
            df['net_flow_score'] * 3 +
            df['price_change_score'] * 2 +
            df['volume_score'] * 1.5 +
            df['balanced_ratio_score'] * 2 +
            df['volatility_score'] * 2
        )

        df['rank'] = df['sirifi_ranking_score'].rank(ascending=False).astype(int)
        df = df.sort_values('rank')

        # Select columns to show
        return df[[
            'rank',
            'ticker',
            'current_price',
            'total_sell_volume',
            'total_buy_volume',
            'buy_sell_diff',
            'total_volume',
            'net_flow_pct',
            'buy_sell_ratio',
            'balanced_ratio_score',
            'price_change_pct',
            'volatility',
            'sirifi_ranking_score'
        ]].reset_index(drop=True)


# 🔧 Example Usage
if __name__ == "__main__":
    API_KEY = "QoHrpRcui2X7py9whGmGmTZyi1ZR7OQrNYmUOC2A7kHt5yx0vQ1sL2o8x8Pjztd8"
    API_SECRET = "fniX66D2I37jS4BwCmV4m6uod6zQLMkP3pPgMtDKHFV76Wihmu85UfhSU843IbyC"


    analyzer = CryptoFlowAnalyzer(
        api_key=API_KEY,           # Your Binance API key (required for authenticated data access)
        api_secret=API_SECRET,     # Your Binance API secret key

        quote_asset='USDC',        # The stablecoin used as the quote currency (e.g., USDC, USDT, BUSD)
        
        timeframe='1h',            # Time window for analysis: '30m', '1h', or '1d' (e.g., 1-hour recent trade data)
        
        max_symbols=50,            # Maximum number of trading pairs to analyze (top 50 with USDC quote)
        
        threads=10,                # Number of concurrent threads used for fetching/analyzing data (for speed)
        
        small_thresh=10,           # Trades <$10 are considered small orders
        mid_thresh=100             # Trades between $10 and <$100 are mid-sized; ≥$100 are large
    )

    df = analyzer.create_summary()
    print(df)

    # Optionally save to CSV
    # df.to_csv('crypto_flow_ranking.csv', index=False)


🔍 Analyzing 50 symbols on 1h timeframe...
[1/50] Processed LTCUSDC
[2/50] Processed XLMUSDC
[3/50] Processed TRXUSDC
[4/50] Processed NEOUSDC
[5/50] Processed BTCUSDC
[6/50] Processed ETCUSDC
[7/50] Processed ATOMUSDC
[8/50] Processed ALGOUSDC
[9/50] Processed ADAUSDC
[10/50] Processed ONTUSDC
[11/50] Processed BCHUSDC
[12/50] Processed ETHUSDC
[13/50] Processed LINKUSDC
[14/50] Processed AVAXUSDC
[15/50] Processed ARBUSDC
[16/50] Processed DOGEUSDC
[17/50] Processed DOTUSDC
[18/50] Processed INJUSDC
[19/50] Processed ORDIUSDC
[20/50] Processed OPUSDC
[21/50] Processed SOLUSDC
[22/50] Processed TIAUSDC
[23/50] Processed BLURUSDC
[24/50] Processed ALTUSDC
[25/50] Processed SEIUSDC
[26/50] Processed JUPUSDC
[27/50] Processed FILUSDC
[28/50] Processed MANTAUSDC
[29/50] Processed XRPUSDC
[30/50] Processed BNBUSDC
[31/50] Processed WLDUSDC
[32/50] Processed SUIUSDC
[33/50] Processed UNIUSDC
[34/50] Processed PIXELUSDC
[35/50] Processed STRKUSDC
[36/50] Processed SHIBUSDC
[37/50] Processed N

In [20]:
df

,rank,ticker,current_price,total_sell_volume,total_buy_volume,buy_sell_diff,total_volume,net_flow_pct,buy_sell_ratio,balanced_ratio_score,price_change_pct,volatility,sirifi_ranking_score
0,1,DOGEUSDC,0.236300,437146.24,567431.46,130285.22,1004577.70,0.1297,1.2980,0.7020,11.23,0.010637,7.6340
1,2,NEARUSDC,2.671000,91773.44,121088.57,29315.13,212862.01,0.1377,1.3194,0.6806,10.51,0.007844,7.6012
2,3,XRPUSDC,3.033800,6956970.33,7085702.67,128732.34,14042673.00,0.0092,1.0185,0.9815,7.58,0.008752,7.2530
3,4,ETHUSDC,4721.080000,9080089.49,8498910.17,-581179.32,17578999.66,-0.0331,0.9360,0.9360,10.28,0.010393,7.2120
4,5,PIXELUSDC,0.035840,3000.42,3462.08,461.66,6462.51,0.0714,1.1539,0.8461,12.60,0.008380,7.1822
5,6,BTCUSDC,115544.000000,4846241.83,6041157.46,1194915.63,10887399.30,0.1098,1.2466,0.7534,2.92,0.004838,7.1468
6,7,BNBUSDC,890.080000,1428766.94,1682969.71,254202.78,3111736.65,0.0817,1.1779,0.8221,5.18,0.005185,7.1042
7,8,SEIUSDC,0.320000,105441.64,105585.80,144.16,211027.43,0.0007,1.0014,0.9986,11.65,0.010944,7.0272
8,9,SOLUSDC,202.730000,3430379.64,3583936.16,153556.53,7014315.80,0.0219,1.0448,0.9552,12.47,0.017312,6.9804
9,10,ADAUSDC,0.914300,1642966.47,2282229.64,639263.17,3925196.11,0.1629,1.3891,0.6109,8.94,0.011462,6.9318


In [21]:
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from binance.client import Client
from concurrent.futures import ThreadPoolExecutor, as_completed


class CryptoValueInvestor:
    def __init__(self, api_key, api_secret, quote_asset='USDC', timeframe='1d', max_symbols=50, threads=10, history_days=90):
        self.client = Client(api_key, api_secret)
        self.quote_asset = quote_asset
        self.timeframe = timeframe
        self.max_symbols = max_symbols
        self.threads = threads
        self.history_days = history_days
        self.timeframe_map = {
            '30m': timedelta(minutes=30),
            '1h': timedelta(hours=1),
            '1d': timedelta(days=1),
        }

    def get_start_time_ms(self):
        delta = self.timeframe_map[self.timeframe]
        return int((datetime.utcnow() - delta).timestamp() * 1000)

    def fetch_all_symbols(self):
        info = self.client.get_exchange_info()
        return [
            s['symbol'] for s in info['symbols']
            if s['quoteAsset'] == self.quote_asset and s['status'] == 'TRADING'
        ][:self.max_symbols]

    def fetch_price_and_market_cap(self, symbol):
        try:
            ticker = self.client.get_ticker(symbol=symbol)
            price = float(ticker.get('lastPrice', 0))
            quote_volume = float(ticker.get('quoteVolume', 0))
            market_cap = quote_volume / 0.05  # Approximate
            return price, float(ticker.get('priceChangePercent', 0)), market_cap
        except:
            return 0.0, 0.0, 0.0

    def fetch_agg_trades(self, symbol, start_time, end_time):
        trades = []
        while True:
            try:
                batch = self.client.get_aggregate_trades(
                    symbol=symbol,
                    startTime=start_time,
                    endTime=end_time,
                    limit=1000
                )
                if not batch:
                    break
                trades.extend(batch)
                last_trade_time = batch[-1]['T']
                if last_trade_time >= end_time:
                    break
                start_time = last_trade_time + 1
                time.sleep(0.05)
            except:
                break
        return trades

    def calculate_historical_metrics(self, symbol):
        try:
            end_time = datetime.utcnow()
            start_time = end_time - timedelta(days=self.history_days)
            klines = self.client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY,
                                                       start_time.strftime("%d %b %Y %H:%M:%S"),
                                                       end_time.strftime("%d %b %Y %H:%M:%S"))

            df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                               'close_time', 'quote_asset_volume', 'num_trades',
                                               'taker_buy_base', 'taker_buy_quote', 'ignore'])
            df['close'] = df['close'].astype(float)
            df['returns'] = df['close'].pct_change()

            # CAGR
            cagr = (df['close'].iloc[-1] / df['close'].iloc[0]) ** (365 / len(df)) - 1

            # Sharpe
            sharpe = df['returns'].mean() / df['returns'].std() * np.sqrt(365) if df['returns'].std() != 0 else 0

            # Max Drawdown
            cumulative = (1 + df['returns']).cumprod()
            drawdown = cumulative / cumulative.cummax() - 1
            max_drawdown = drawdown.min()

            return round(cagr, 4), round(sharpe, 4), round(max_drawdown, 4)

        except Exception as e:
            print(f"[!] Historical data error for {symbol}: {e}")
            return 0.0, 0.0, 0.0

    def process_symbol(self, symbol, start_time, end_time):
        trades = self.fetch_agg_trades(symbol, start_time, end_time)
        if not trades:
            return None

        total_buy = 0
        total_sell = 0
        for trade in trades:
            value = float(trade['p']) * float(trade['q'])
            if trade['m']:
                total_sell += value
            else:
                total_buy += value

        total_volume = total_buy + total_sell
        net_flow_pct = (total_buy - total_sell) / total_volume if total_volume > 0 else 0
        price, price_change_pct, market_cap = self.fetch_price_and_market_cap(symbol)

        cagr, sharpe, max_dd = self.calculate_historical_metrics(symbol)

        return {
            'symbol': symbol,
            'price': round(price, 4),
            'price_change_pct': price_change_pct,
            'total_volume': round(total_volume, 2),
            'net_flow_pct': round(net_flow_pct, 4),
            'market_cap': round(market_cap, 2),
            'cagr': cagr,
            'sharpe': sharpe,
            'max_drawdown': max_dd
        }

    def analyze(self):
        symbols = self.fetch_all_symbols()
        start_time = self.get_start_time_ms()
        end_time = int(datetime.utcnow().timestamp() * 1000)
        print(f"Analyzing {len(symbols)} symbols with historical data...")

        results = []
        with ThreadPoolExecutor(max_workers=self.threads) as executor:
            futures = {executor.submit(self.process_symbol, symbol, start_time, end_time): symbol for symbol in symbols}
            for future in as_completed(futures):
                data = future.result()
                if data:
                    results.append(data)

        df = pd.DataFrame(results)
        if not df.empty:
            df['volume_yield'] = df['total_volume'] / df['market_cap']
            df['value_score'] = (
                (1 - df['price_change_pct'].rank(pct=True)) * 1.5 +  # Contrarian
                df['net_flow_pct'].rank(pct=True) * 1.0 +
                df['volume_yield'].rank(pct=True) * 1.0 +
                df['cagr'].rank(pct=True) * 2.0 +
                df['sharpe'].rank(pct=True) * 1.5 +
                (1 - df['max_drawdown'].rank(pct=True)) * 1.0  # Lower is better
            )
            df['rank'] = df['value_score'].rank(ascending=False).astype(int)
            df = df.sort_values(by='rank')

        return df[['rank', 'symbol', 'price', 'price_change_pct', 'net_flow_pct', 'total_volume',
                   'market_cap', 'volume_yield', 'cagr', 'sharpe', 'max_drawdown', 'value_score']].reset_index(drop=True)


In [22]:
if __name__ == "__main__":
   

    investor = CryptoValueInvestor(
        api_key=API_KEY,
        api_secret=API_SECRET,
        quote_asset='USDC',
        timeframe='1d',
        max_symbols=20,
        threads=10,
        history_days=90  # Use 90-day price history
    )

    df = investor.analyze()
    df
    # Optionally save: df.to_csv("crypto_value_ranking.csv", index=False)


Analyzing 20 symbols with historical data...


,rank,symbol,price,price_change_pct,net_flow_pct,total_volume,market_cap,volume_yield,cagr,sharpe,max_drawdown,value_score
0,1,LINKUSDC,25.5500,5.491,-0.0074,2.472681e+08,4.785652e+09,0.051669,6.5120,2.6767,-0.2676,6.3000
1,2,XLMUSDC,0.4098,6.386,0.0220,1.675531e+07,3.315650e+08,0.050534,3.3064,2.0346,-0.2626,5.7500
2,3,BNBUSDC,886.3200,4.403,0.0963,2.034128e+08,4.060690e+09,0.050093,2.0268,3.1016,-0.1280,5.6250
3,4,ARBUSDC,0.5828,20.363,0.0332,2.557328e+07,5.069358e+08,0.050447,3.9740,1.9375,-0.3538,5.2500
4,5,BCHUSDC,584.4000,5.316,0.0323,6.044184e+06,1.210423e+08,0.049934,2.9766,2.5257,-0.1188,5.1500
5,6,LTCUSDC,120.9500,6.050,0.1311,1.077948e+07,2.135269e+08,0.050483,1.6561,1.8615,-0.1637,5.1000
6,7,TRXUSDC,0.3612,1.775,-0.0019,1.715876e+07,3.460795e+08,0.049580,2.0666,3.7087,-0.1001,5.0500
7,7,ETHUSDC,4704.4200,10.554,-0.0156,1.929934e+09,3.855807e+10,0.050053,10.7052,3.8632,-0.2089,5.0500
8,9,XRPUSDC,3.0352,8.446,-0.0127,8.405786e+08,1.629603e+10,0.051582,2.0215,1.9313,-0.2205,4.6500
9,10,ALGOUSDC,0.2600,7.128,0.0797,6.479680e+06,1.304011e+08,0.049690,1.0020,1.1937,-0.2864,4.5500


In [19]:
from binance.client import Client
from newsapi import NewsApiClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

class CryptoSentimentAnalyzer:
    def __init__(self, binance_api_key, binance_api_secret, newsapi_key, 
                 quote_asset='USDC', max_symbols=100, threads=5):
        self.binance_client = Client(binance_api_key, binance_api_secret)
        self.newsapi = NewsApiClient(api_key=newsapi_key)
        self.analyzer = SentimentIntensityAnalyzer()
        self.quote_asset = quote_asset
        self.max_symbols = max_symbols
        self.threads = threads
        
        # Will hold symbol-to-fullname mapping
        self.symbol_name_map = {}

    def fetch_symbols_and_names(self):
        info = self.binance_client.get_exchange_info()
        symbols = []
        for s in info['symbols']:
            if s['quoteAsset'] == self.quote_asset and s['status'] == 'TRADING':
                symbols.append(s['symbol'])
                self.symbol_name_map[s['symbol']] = s['baseAsset']
        return symbols[:self.max_symbols]

    def get_sentiment_for_news(self, query):
        """
        Fetch news articles for the query, and compute aggregate sentiment.
        Returns dict with average positive, negative, neutral, compound, and total articles.
        """
        try:
            all_articles = []
            # Fetch up to 100 articles in multiple pages (NewsAPI allows max 100 per request)
            for page in range(1, 3):  # try two pages max to limit requests
                response = self.newsapi.get_everything(
                    q=query,
                    language='en',
                    sort_by='relevancy',
                    page_size=50,
                    page=page
                )
                if 'articles' not in response or not response['articles']:
                    break
                all_articles.extend(response['articles'])
                if len(response['articles']) < 50:
                    break
                time.sleep(1)  # be polite to API rate limits
            
            if not all_articles:
                return None
            
            # Aggregate sentiment scores
            pos, neg, neu, comp = 0, 0, 0, 0
            for article in all_articles:
                text = (article.get('title') or '') + ' ' + (article.get('description') or '')
                vs = self.analyzer.polarity_scores(text)
                pos += vs['pos']
                neg += vs['neg']
                neu += vs['neu']
                comp += vs['compound']
            
            count = len(all_articles)
            return {
                'positive': pos / count,
                'negative': neg / count,
                'neutral': neu / count,
                'compound': comp / count,
                'total_articles': count
            }
        except Exception as e:
            print(f"[!] NewsAPI error for query '{query}': {e}")
            return None

    def fetch_price(self, symbol):
        try:
            ticker = self.binance_client.get_symbol_ticker(symbol=symbol)
            return float(ticker['price'])
        except:
            return None

    def analyze_single_crypto(self, symbol):
        """
        Runs sentiment for both symbol and full coin name, merges results,
        and gets current price.
        """
        full_name = self.symbol_name_map.get(symbol, '')
        # Query both symbol and full name
        sentiment_symbol = self.get_sentiment_for_news(symbol)
        sentiment_name = self.get_sentiment_for_news(full_name)

        # Combine sentiment scores if both available
        def combine_sentiments(a, b):
            if a and b:
                return {
                    k: (a[k] + b[k]) / 2 for k in a.keys()
                }
            return a or b

        combined_sentiment = combine_sentiments(sentiment_symbol, sentiment_name)
        if combined_sentiment is None:
            return None

        price = self.fetch_price(symbol)
        if price is None:
            return None

        # Calculate ranking score: weighted compound + positive/negative ratio
        pos_neg_ratio = combined_sentiment['positive'] / (combined_sentiment['negative'] + 1e-6)
        ranking_score = 0.6 * combined_sentiment['compound'] + 0.4 * pos_neg_ratio

        return {
            'symbol': symbol,
            'name': full_name,
            'price': round(price, 6),
            'positive': round(combined_sentiment['positive'], 4),
            'negative': round(combined_sentiment['negative'], 4),
            'neutral': round(combined_sentiment['neutral'], 4),
            'compound': round(combined_sentiment['compound'], 4),
            'total_articles': combined_sentiment['total_articles'],
            'pos_neg_ratio': round(pos_neg_ratio, 4),
            'ranking_score': round(ranking_score, 4)
        }

    def analyze_all(self):
        symbols = self.fetch_symbols_and_names()
        print(f"Analyzing {len(symbols)} cryptos for sentiment...")

        results = []
        with ThreadPoolExecutor(max_workers=self.threads) as executor:
            futures = {executor.submit(self.analyze_single_crypto, sym): sym for sym in symbols}
            for i, future in enumerate(as_completed(futures), 1):
                sym = futures[future]
                try:
                    res = future.result()
                    if res:
                        results.append(res)
                except Exception as e:
                    print(f"[!] Error processing {sym}: {e}")
                print(f"[{i}/{len(symbols)}] Processed {sym}")

        df = pd.DataFrame(results)
        if not df.empty:
            df.sort_values('ranking_score', ascending=False, inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.index += 1  # Start index at 1 for ranking display
            df.rename_axis('Rank', inplace=True)
        return df

# Usage example
if __name__ == "__main__":
    BINANCE_API_KEY = "QoHrpRcui2X7py9whGmGmTZyi1ZR7OQrNYmUOC2A7kHt5yx0vQ1sL2o8x8Pjztd8"
    BINANCE_API_SECRET = "fniX66D2I37jS4BwCmV4m6uod6zQLMkP3pPgMtDKHFV76Wihmu85UfhSU843IbyC"


    NEWSAPI_KEY = "cfdf9bd9c88c443c97e171e3d7f6d41d"

    analyzer = CryptoSentimentAnalyzer(
        binance_api_key=BINANCE_API_KEY,
        binance_api_secret=BINANCE_API_SECRET,
        newsapi_key=NEWSAPI_KEY,
        quote_asset='USDC',
        max_symbols=50,
        threads=5
    )

    sentiment_df = analyzer.analyze_all()
    print(sentiment_df)
    # sentiment_df.to_csv('crypto_sentiment_ranking.csv')


Analyzing 50 cryptos for sentiment...
[!] NewsAPI error for query 'XLMUSDC': {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}[!] NewsAPI error for query 'BNBUSDC': {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}
[!] NewsAPI error for query 'ETHUSDC': {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

[!] NewsAPI error for query 'XRPUSDC': {'status': 

In [16]:
sentiment_df

""
